
### 🌿 1. Seq2Seq란?

#### 📌 정의
입력 시퀀스(문장)를 받아 **출력 시퀀스(새 문장)** 을 생성하는 모델 구조.

- 영어 문장을 한국어로 번역  
- 질문 → 답변 생성  
- 긴 문장 → 요약문 생성  

자연어 생성 작업에서 가장 기본적으로 사용되는 딥러닝 구조입니다.

#### 📌 왜 중요한가?
자연어는 **앞·뒤 문맥에 크게 의존**합니다.  
하지만 전통적인 분류 모델이나 고정 벡터 기반 모델은 이 문맥 흐름을 충분히 반영하지 못합니다.

그래서 등장한 것이 **Seq2Seq 모델**  
→ 시퀀스 전체를 읽고, 그 의미를 바탕으로 새로운 시퀀스를 **순차적으로 생성**합니다.

---

### ⚙️ 2. Seq2Seq가 필요한 이유

| 문제점 | 설명 |
|-------|------|
| **입력/출력 길이가 서로 다름** | 번역: "I love you" → "나는 너를 사랑해" (길이가 다름) |
| **토큰 간 문맥 의존성 무시** | 단순 모델은 앞뒤 단어 영향 반영이 어렵다 |
| **순서 정보 손실** | 고정 피처 벡터는 문장 흐름을 담기 어렵다 |
| **생성 문제에 부적합** | 분류 모델은 단어를 하나씩 생성하는 작업에 맞지 않음 |

---

### 🧠 3. Seq2Seq 동작 원리

#### ① **Encoder — 입력 문장 읽고 요약**
- LSTM/GRU가 입력 문장의 토큰을 순서대로 처리  
- 마지막 Hidden State 단계에 문장 전체 의미(요약 정보) 저장

#### ② **Context Vector — 문장의 요약본**
- Encoder가 만든 문장 의미를 하나의 벡터로 !! 압축 !!
- Decoder에게 전달  
- 번역 기준으로는 “원문을 완전히 이해한 상태”

#### ③ **Decoder — 새로운 문장 생성**
- Context를 첫 입력으로 받아 다음 단어를 하나씩 생성  
- 이전에 생성한 단어를 다시 입력으로 넣으며 문장을 이어감

#### ④ **EOS (End Of Sequence) — 종료 신호**
- Decoder가 EOS 토큰을 만들면 문장 생성 종료  
- “여기서 문장이 끝났어!”라는 신호

---

### 💡 4. 실제 활용 예시

#### 🔸 1) 기계 번역 (Machine Translation)
- 영어 → 한국어  
- 한국어 → 일본어  
→ 가장 대표적인 Seq2Seq 활용 분야

#### 🔸 2) 대화 시스템 (Chatbot / QA)
- 질문 입력 → 자연스러운 답변 생성  
- 고객센터 챗봇, AI 스피커 등

#### 🔸 3) 문서 요약 (Summarization)
- 긴 문장/기사 → 핵심 요약  
- 뉴스 요약, 보고서 자동 요약 시스템

#### 🔸 4) 코드 자동 생성 (Code Generation)
- "숫자 리스트 정렬하는 코드 작성해줘" → 코드 출력  
- GitHub Copilot, ChatGPT 구조에 사용되는 핵심 아이디어

---


In [ ]:
# 문자 인덱싱 및 원-핫 인코딩
# 입력과 타겟의 고유한 문자 수집
# input_characters = set()
# target_characters = set()

# for text in input_texts:
#     for char in text:
#         input_characters.add(char)

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# =========================
# 1. 문자 집합(vocabulary) 만들기
# =========================

# input_texts 안에 있는 모든 문장에서 문자 하나하나를 모아서 집합으로 만든다.
# { ... } 컴프리헨션: 중복 제거 + 집합(Set) 생성
input_characters = {char for text in input_texts for char in text}

# target_texts 안에 있는 모든 문장에서 문자 하나하나를 모아서 집합으로 만든다.
target_characters = {char for target_text in target_texts for char in target_text}

# 집합(Set)은 순서가 없으니까, sorted()로 정렬해서 리스트로 만든다.
# 이렇게 해야 나중에 index가 항상 일관되게 유지된다.
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# 인코더에서 사용할 문자 개수 (문자 사전 크기)
num_encoder_tokens = len(input_characters)

# 디코더에서 사용할 문자 개수 (문자 사전 크기)
num_decoder_tokens = len(target_characters)

# =========================
# 2. 시퀀스(문장) 길이 정보
# =========================

# input_texts 중에서 가장 긴 문장의 길이를 찾는다. (인코더 최대 길이)
max_encoder_seq_length = max(len(txt) for txt in input_texts)

# target_texts 중에서 가장 긴 문장의 길이를 찾는다. (디코더 최대 길이)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

# =========================
# 3. 문자 ↔ 인덱스 매핑 딕셔너리
# =========================

# 문자 -> 인덱스 : enumerate로 0부터 순서대로 번호를 붙인다.
# 예: {'a': 0, 'b': 1, ...}
input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}

# 인덱스 -> 문자 : 추론(inference)할 때 숫자를 다시 문자로 바꾸기 위해 역매핑을 만든다.
# 딕셔너리 뒤집기 (key, value 위치 바꾸기)
reverse_input_token_index = {idx: char for char, idx in input_token_index.items()}
reverse_target_token_index = {idx: char for char, idx in target_token_index.items()}

# =========================
# 4. 원-핫 인코딩(One-Hot Encoding)용 배열 준비
# =========================

# encoder_input_data: 형태 = (샘플 수, 인코더 최대 길이, 인코더 문자 수)
# 처음에는 모두 0으로 채운다. (나중에 특정 위치만 1.0으로 채운다)
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32'
)

# decoder_input_data: 형태 = (샘플 수, 디코더 최대 길이, 디코더 문자 수)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32'
)

# decoder_target_data: 디코더가 한 스텝 뒤를 맞추도록 하는 "정답" 데이터
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32'
)

# =========================
# 5. 실제 원-핫 인코딩 채우기 + Teacher Forcing용 타깃 만들기
# =========================

# i: 샘플 인덱스, input_text: 입력 문장, target_text: 타깃 문장
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # ---------- 인코더 입력 ----------
    for t, char in enumerate(input_text):
        # i번째 문장, t번째 타임스텝, 해당 문자 위치 인덱스를 1.0으로 설정
        # 즉, "이 위치에는 이 문자다" 라고 표시하는 것
        encoder_input_data[i, t, input_token_index[char]] = 1.0

    # ---------- 디코더 입력 & 타깃 ----------
    for t, char in enumerate(target_text):
        # decoder_input_data: 디코더가 "입력으로" 받는 문자
        # 예) '\t안녕\n' 전체를 그대로 넣어줌
        decoder_input_data[i, t, target_token_index[char]] = 1.0

        # decoder_target_data: 디코더가 "맞춰야 할 정답" (한 스텝 앞)
        # 예시)
        #   디코더 입력 :   \t 안 녕 \n
        #   디코더 타깃 :   안 녕 \n (한 칸 오른쪽으로 밀린 것)
        # 그래서 t > 0 일 때, t-1 위치에 현재 char를 정답으로 둔다.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
# hi  입력
# \t hello  디코더 입력
# hello \n  디코더 출력 - Teacher Forcing  한스텝 앞으로 이동
print(f'고유 입력 문자수 : {num_encoder_tokens}')
print(f'고유 타겟 문자수 : {num_decoder_tokens}')
print(f'최대 입력 문장길이 : {max_encoder_seq_length}')
print(f'최대 타겟 문장길이 : {max_decoder_seq_length}')
print('# 샘플, 시퀀스 길이, 문자 사전 크기')
print(f'encoder_input_data : {encoder_input_data.shape}')
print(f'decoder_input_data : {decoder_input_data.shape}')
print(f'decoder_target_data : {decoder_target_data.shape}')

# =========================
# 6. 모델 설정을 위한 차원(은닉 상태 크기)
# =========================

latent_dim = 256  # LSTM 은닉 상태 크기 (내부 메모리 크기, 특징 벡터 차원 수)

# =========================
# 7. Encoder (인코더) 정의
# =========================

# 인코더 입력 텐서 정의
# shape=(None, num_encoder_tokens)
#  - None: 타임스텝 길이(문장 길이). 가변 길이 시퀀스 허용.
#  - num_encoder_tokens: 한 타임스텝에서 들어오는 원-핫 벡터 길이(문자 수)
encoder_inputs = Input(shape=(None, num_encoder_tokens), name='encoder_input')

# LSTM 층 정의
# return_state=True → 마지막 시점의 hidden state(h)와 cell state(c)를 같이 반환
encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')

# encoder_outputs: 모든 타임스텝의 출력 (여기선 사용 안 함)
# state_h: 마지막 타임스텝의 hidden state
# state_c: 마지막 타임스텝의 cell state
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# 디코더에 넘겨줄 인코더의 최종 상태(h, c)
encoder_states = [state_h, state_c]

# =========================
# 8. Decoder (디코더) 정의
# =========================

# 디코더 입력 텐서 정의
# shape=(None, num_decoder_tokens)
#   - None: 디코더 시퀀스 길이
#   - num_decoder_tokens: 타겟 문자 사전 크기
decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_input')

# 디코더 LSTM 정의
# return_sequences=True → 모든 타임스텝의 출력을 반환 (각 타임스텝마다 softmax 하기 위해)
# return_state=True → 마지막 상태도 같이 반환 (추론용에 필요)
decoder_lstm = LSTM(
    latent_dim,
    return_sequences=True,
    return_state=True,
    name='decoder_lstm'
)

# 디코더 LSTM 실행
# initial_state=encoder_states → 인코더의 마지막 상태를 디코더의 초기 상태로 사용
# 즉, "입력 문장의 의미"를 디코더에게 넘겨주는 역할
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# =========================
# 9. 출력층(Dense + softmax)
# =========================

# 각 타임스텝에서 "다음에 올 문자"에 대한 확률 분포를 뽑는 Dense 층
# num_decoder_tokens 크기의 벡터를 출력하고, softmax로 확률 분포가 됨
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')

# 디코더의 RNN 출력(decoder_outputs)에 Dense 층을 씌워 최종 문자 확률을 얻는다.
decoder_outputs = decoder_dense(decoder_outputs)

# =========================
# 10. 학습용 전체 모델 정의
# =========================

# 입력: [encoder_inputs, decoder_inputs]
# 출력: decoder_outputs (각 타임스텝마다의 문자 확률)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs, name='seq2seq_training')

print("\n 모델 구조:")
model.summary()


In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense


data_pairs = [
    ("Hello", "Bonjour"),
    ("How are you", "Comment allez-vous"),
    ("Good morning", "Bonjour matin"),
    ("Thank you", "Merci"),
]

# 입력과 타겟을 분리
input_texts = []
target_texts = []
for eng , fra in data_pairs:
    input_texts.append(eng)
    target_texts.append(f'\t{fra}\n')
for i in range(len(input_texts)):
    print(f'입력 : {input_texts[i]:20s} --> 타겟 : {target_texts[i]}')

입력 : Hello                --> 타겟 : 	Bonjour

입력 : How are you          --> 타겟 : 	Comment allez-vous

입력 : Good morning         --> 타겟 : 	Bonjour matin

입력 : Thank you            --> 타겟 : 	Merci




문자 단위 사전(vocabulary) 생성 및 정수 인덱스 변환

개념:
    
각 문자를 고유한 정수로 매핑

입력과 타겟의 사전은 별도 관리

원-핫 인코딩으로 신경망 입력 형태 생성

설명:
    
input_characters: 영어 문장에 등장하는 모든 고유 문자

target_characters: 프랑스어 문장 + 특수 토큰(\t, \n)

encoder_input_data: 3D 배열 (샘플, 시퀀스 길이, 문자 사전 크기)


In [14]:


input_characters = {char for text in input_texts for char in text}
target_characters = {char for target_text in target_texts  for char in target_text}

# 정렬해서 일관성 확보
input_characters = sorted(list(input_characters))
target_characters= sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# 가장 긴 문장 길이 계산
max_encoder_seq_length = max(len(txt) for txt in input_texts  )
max_decoder_seq_length = max(len(txt) for txt in target_texts  )
# 문자-> 인덱스 매핑
input_token_index = { char:i for i,char in enumerate(input_characters)}
target_token_index = { char:i for i,char in enumerate(target_characters)}

# 인덱스 -> 문자 역매핑(추론시 사용)
reverse_input_token_index =  { idx:char for char,idx in input_token_index.items()}
reverse_target_token_index = { idx:char for char,idx in target_token_index.items()}
#  encoder_input_data: 3D 배열 (샘플, 시퀀스 길이, 문자 사전 크기)
encoder_input_data = np.zeros( (len(input_texts),max_encoder_seq_length,num_encoder_tokens ),
                            dtype='float32' )
decoder_input_data = np.zeros( (len(input_texts),max_decoder_seq_length,num_decoder_tokens ),
                            dtype='float32' )
decoder_target_data = np.zeros( (len(input_texts),max_decoder_seq_length,num_decoder_tokens ),
                            dtype='float32' )

# 문자별 원핫 인코딩
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0    
    for t, char in enumerate(target_text):
        # decoder_input_data: 전체 타겟 시퀀스 (시작 토큰 포함)
        decoder_input_data[i, t, target_token_index[char]] = 1.0        
        # decoder_target_data: 한 타임스텝 앞선 정답 (Teacher Forcing용)
        # 디코더 입력 \t안녕
        # 디코더 출력 안녕\n
        # 한 스텝 시프트 -  Teacher Forcing
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0



In [15]:
latent_dim = 256  # LSTM 은닉 차원 (내부 표현 크기)

# ==================== Encoder ====================
encoder_inputs = Input(shape=(None, num_encoder_tokens), name='encoder_input')
encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# encoder_outputs는 사용하지 않고, 내부 상태(state_h, state_c)만 디코더로 전달
# 입력 시퀀스를 LSTM에 통과시켜서 마지막 은닉상태state_h)와 셀상태(state_c)를 받아서
# 두 상태를 입력문장의 의미(context)를 압축한 벡터
encoder_states = [state_h, state_c]

# ==================== Decoder ====================
decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_input')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')

# 디코더 초기 상태로 인코더 최종 상태 사용 (컨텍스트 전달)
# 인코더의 상태(state_h , state_c)를 초기상태로 받아서 자신의 입력 decoder_inputs을 기반으로 다음 단어를 예측
# 각 시점의 출력은 Dense+Softmax거쳐서 단어(문자) 확률분포
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# 각 타임스텝에서 문자 확률 분포 생성
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

# ==================== 학습 모델 ====================
model = Model([encoder_inputs, decoder_inputs], decoder_outputs, name='seq2seq_training')

print("\n 모델 구조:")
model.summary()


 모델 구조:


Model: "seq2seq_training"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, None, 19)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input       │ (None, None, 22)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 256),     │    282,624 │ encoder_input[0]… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    285,696 │ decoder_input[0]… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 22)  │      5,654 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 573,974 (2.19 MB)

 Trainable params: 573,974 (2.19 MB)

 Non-trainable params: 0 (0.00 B)

목적: Seq2Seq 모델 컴파일 및 학습 실행

개념:

    - categorical_crossentropy: 다중 클래스(문자 사전) 손실

    - Teacher Forcing: decoder_input_data는 정답 시퀀스 전체 제공

    - 학습 목표: decoder_target_data (한 타임스텝 앞당긴 정답)

설명:

    - optimizer='rmsprop': 순환신경망에 안정적인 최적화 알고리즘

    - epochs=100: 작은 데이터셋이므로 충분한 반복 필요

    - batch_size=2: 메모리 효율 (실제로는 전체 4개 샘플 사용)

In [16]:
model.compile(
    optimizer = 'rmsprop',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    [encoder_input_data,decoder_input_data],  # seq2seq
    decoder_target_data,
    batch_size = 2,
    epochs = 10000,
    # vallidation_split = 0.0,  # 데이터셋이 작아서 분할 안함
    verbose = 0 # 0 출력안하고 1은 간단하게 2 좀더 간단하게 출력
)

In [17]:
history.history['accuracy'][-1]

0.550000011920929

In [18]:
# Encoder 추론 모델
encoder_model = Model(encoder_inputs,encoder_states , name='encoder_inference')

In [20]:
# Decoder 추론 모델
# 이전 타임스텝의 상태를 입력으로 받음
decoder_state_input_h = Input(shape=(latent_dim,) , name='decoder_state_h')
decoder_state_input_c = Input(shape=(latent_dim,) , name='decoder_state_c')
decoder_state_inputs = [decoder_state_input_h , decoder_state_input_c]

# LSTM 실행 (이전상태 + 현재입력)
decoder_outputs , state_h , state_c = decoder_lstm(decoder_inputs,initial_state=decoder_state_inputs)
decoder_states = [state_h , state_c]

# 문자 확률 분포 생성
decoder_outputs = decoder_dense(decoder_outputs)

# 모델 조립
decoder_model = Model(
    [decoder_inputs] + decoder_state_inputs,
    [decoder_outputs] + decoder_states,
    name='decoder_inference'
)

입력 문장을 번역하는 decoding 함수 구현 및 테스트

개념:
    - Greedy Decoding: 매 스텝 가장 높은 확률 문자 선택

    - 종료 조건: '\n' 토큰 생성 또는 최대 길이 도달
    
    - 자기회귀적 생성: 이전 예측을 다음 입력으로 반복 사용

설명:

    1. Encoder로 입력 문장의 상태 벡터 추출

    2. 시작 토큰('\t')으로 Decoder 시작

    3. 반복: 현재 문자 예측 → 다음 입력으로 사용

    4. '\n' 만나면 종료

In [30]:
def decode_sequence(input_seq):
    """
    입력 시퀀스(원-핫 인코딩)를 받아 번역된 문자열 반환
    """
    # 1단계: Encoder로 상태 벡터 추출
    states_value = encoder_model.predict(input_seq, verbose=0)
    
    # 2단계: 디코더 시작 토큰 준비 ('\t')
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.0
    
    # 3단계: 문자를 하나씩 생성
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        # 현재 문자 예측 + 다음 상태 업데이트
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )
        
        # 가장 높은 확률의 문자 선택 (Greedy)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_token_index[sampled_token_index]
        
        # 문자 추가
        decoded_sentence += sampled_char
        
        # 종료 조건 체크
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        
        # 다음 스텝 준비: 현재 예측을 다음 입력으로
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        
        # 상태 업데이트
        states_value = [h, c]
    
    return decoded_sentence

In [31]:
for seq_index in range(len(input_texts)):
    # 원 핫 인코딩 입력 추출
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    
    # 시작/종료 토큰 제거
    decoded_sentence = decoded_sentence.replace('\t','').replace('\n','')
    print(f'입력 문장 : {input_texts[seq_index]}')
    print(f'정답 문장 : {target_texts[seq_index][1:-1]}') # 시작/종료 토큰을 제거
    print(f'모델이 예측한 결과 : {decoded_sentence}')
    print

입력 문장 : Hello
정답 문장 : Bonjour
모델이 예측한 결과 : Bonjour
입력 문장 : How are you
정답 문장 : Comment allez-vous
모델이 예측한 결과 : Comment allez-vou
입력 문장 : Good morning
정답 문장 : Bonjour matin
모델이 예측한 결과 : Bonjour mati
입력 문장 : Thank you
정답 문장 : Merci
모델이 예측한 결과 : Merci
